# 01 — Data Preparation (TMX → parallel text → train/dev/test)

**Purpose:**
 Prepare clean, aligned parallel data in the exact tab-separated format expected by NLLB and Hugging Face Datasets.

**Inputs:**

- `data/raw/source/source.txt` → Tagalog sentences.
- `data/raw/target/target.txt` → Cebuano translations.

**Process:**

- Filters short/long lines (for quality).
- Shuffles with the fixed seed for reproducibility.
- Splits into 80/10/10 train/dev/test.
- Writes to `data/processed/` as:
  - `train.tsv`
  - `dev.tsv`
  - `test.tsv`

**Outputs:**

- Aligned pairs of sentences for training and evaluation.
- Data sanity check printed via `df.sample()` (so you can visually confirm alignment).

In [5]:
import os, pathlib, sys
root = pathlib.Path(".")
print("Working directory:", root.resolve())
print("\nExpected files:")
print(" - src/prepare/convert_tmx.py")
print(" - src/prepare/make_splits.py")
print(" - data/raw/ceb-tl.tmx")

Working directory: D:\OneDrive\Documents\My Learning Resource\University Courses\DLSU\2025-26\T1\CSC715M\assignments\mc02\notebooks

Expected files:
 - src/prepare/convert_tmx.py
 - src/prepare/make_splits.py
 - data/raw/ceb-tl.tmx


## A) Convert TMX → `source.txt` & `target.txt`

This uses `lxml` with `recover=True` to tolerate broken XML.

In [6]:
!python ../src/prepare/convert_tmx.py --tmx ../data/raw/ceb-tl.tmx --src_lang tl --tgt_lang ceb --out_dir ../data/raw

Extracted 30,919 aligned pairs
Wrote:
  ..\data\raw\source\source.txt
  ..\data\raw\target\target.txt


## B) Clean & split into train/dev/test

In [13]:
!python ../src/prepare/make_splits.py --src ../data/raw/source/source.txt --tgt ../data/raw/target/target.txt --out ../data/processed


Writing splits...
  Wrote  24734 lines → ..\data\processed\train.tsv
  Wrote   3091 lines → ..\data\processed\dev.tsv
  Wrote   3093 lines → ..\data\processed\test.tsv

Verifying actual pair counts via pandas:
  train.tsv:  22851 actual pairs (pandas-parsed)
  dev.tsv:   2930 actual pairs (pandas-parsed)
  test.tsv:   2750 actual pairs (pandas-parsed)

Total usable pre-split pairs: 30,918


## C) Peek at the splits + quick QC


In [14]:
import pandas as pd, pathlib

pp = pathlib.Path("../data/processed")
for name in ["train.tsv", "dev.tsv", "test.tsv"]:
    df = pd.read_csv(pp / name, sep="\t", header=None, names=["src","tgt"])
    print(f"{name}: {len(df):,} pairs")
    display(df.sample(min(5, len(df))))

# Quick quality checks
train = pd.read_csv(pp/"train.tsv", sep="\t", header=None, names=["src","tgt"])
avg_src_len = train["src"].str.split().map(len).mean()
avg_tgt_len = train["tgt"].str.split().map(len).mean()
ratio = (train["src"].str.len() / train["tgt"].str.len()).clip(upper=10).mean()

print(f"\nAverage token length — SRC: {avg_src_len:.1f}, TGT: {avg_tgt_len:.1f}")
print(f"Average char length ratio (src/tgt, clipped): {ratio:.2f}")


train.tsv: 22,851 pairs


,src,tgt
5207,At sa hain na mga handog tungkol sa kapayapaan...,"Ug alang sa halad sa mga halad-sa-pakigdait, d..."
4,At sila'y lumapit sa harap ni Eleazar na saser...,"Ug mingduol sila sa atubangan ni Eleazar, ang ..."
5045,Sapagka't ang mga bagay na ginagawa nila sa li...,Kay makauulaw gayud ang paghisgot sa mga butan...
5727,Sapagka't tinupad mo ang salita ng aking pagti...,Tungod kay gibantayan mo man ang akong sulti s...
3832,"Iisa ang tagapagbigay ng kautusan at hukom, sa...","Usa ra ang magbabalaod ug maghuhukom, siya nga..."


dev.tsv: 2,930 pairs


,src,tgt
460,"At kaniyang sinabi, Ano nga ang magagawa sa ka...",Ug siya miingon: Nan unsa man ang pagabuhaton ...
1268,"Kung ako nga, na Panginoon at Guro, ay naghuga...",Ug kon ako nga inyong Ginoo ug Magtutudlo nang...
291,At lahat na uwak ayon sa kanilang pagkauwak;,"Ug ang tagsatagsa ka owak; ingon sa iyang matang,"
2815,At nangyari nang ikalimang taon ng haring Robo...,"Ug nahitabo sa ikalima ka tuig ni hari Roboam,..."
2228,"Kundi mangagbalot sila ng kayong magaspang, an...","Hinonoa bistihan ninyo sila sa sako, lakip ang..."


test.tsv: 2,750 pairs


,src,tgt
2375,Nang magkagayo'y nagsitayo sa mga baytang ng m...,Unya mingtindog sa ibabaw sa hagdanan sa mga L...
1287,Si Heber nga na Cineo ay humiwalay sa mga Cine...,Ug si Heber ang Cinehanon mibulag gikan sa mga...
1329,"Sapagka't nang mangako ang Dios kay Abraham, p...","Kay sa pagsaad sa Dios kang Abraham, sanglit w..."
1555,At mula sa panahon na ang palaging handog na s...,Ug gikan sa panahon nga ang dayon nga halad-ng...
1753,"At ibinigay nila sa kanila ang Chiriath-arba, ...","Ug sila minghatag kanila sa Chiriath-arba, kin..."



Average token length — SRC: 26.6, TGT: 32.3
Average char length ratio (src/tgt, clipped): 0.97
